In [1]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy import stats
print ('Ran')

Ran


In [2]:
credit_card_bal = pd.read_csv('D:/Project_Data/credit_card_balance.csv')

In [3]:
credit_card_bal.shape

(3840312, 23)

credit_card_bal=credit_card_bal.iloc[0:1048575,:]

In [4]:
credit_card_bal.shape

(3840312, 23)

In [5]:
cnt_col=[col for col in credit_card_bal.columns if 'CNT' in col]

In [6]:
cnt_col

['CNT_DRAWINGS_ATM_CURRENT',
 'CNT_DRAWINGS_CURRENT',
 'CNT_DRAWINGS_OTHER_CURRENT',
 'CNT_DRAWINGS_POS_CURRENT',
 'CNT_INSTALMENT_MATURE_CUM']

In [7]:
cat_features = [f for f in credit_card_bal.columns if credit_card_bal[f].dtype == 'object' and f not in cnt_col]

In [8]:
cat_features

['NAME_CONTRACT_STATUS']

In [9]:
num_features = [f for f in credit_card_bal.columns if credit_card_bal[f].dtype != 'object' and f not in cnt_col]

In [10]:
num_features

['SK_ID_PREV',
 'SK_ID_CURR',
 'MONTHS_BALANCE',
 'AMT_BALANCE',
 'AMT_CREDIT_LIMIT_ACTUAL',
 'AMT_DRAWINGS_ATM_CURRENT',
 'AMT_DRAWINGS_CURRENT',
 'AMT_DRAWINGS_OTHER_CURRENT',
 'AMT_DRAWINGS_POS_CURRENT',
 'AMT_INST_MIN_REGULARITY',
 'AMT_PAYMENT_CURRENT',
 'AMT_PAYMENT_TOTAL_CURRENT',
 'AMT_RECEIVABLE_PRINCIPAL',
 'AMT_RECIVABLE',
 'AMT_TOTAL_RECEIVABLE',
 'SK_DPD',
 'SK_DPD_DEF']

In [11]:
df = credit_card_bal[num_features]

In [12]:
df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0,0


In [13]:
def agg_numeric(df, group_var):

    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = df.groupby(group_var).agg('mean').reset_index()

    return agg

In [14]:
num_group_df = agg_numeric(df.drop(columns = ['SK_ID_PREV']), group_var = 'SK_ID_CURR')

In [15]:
num_group_df.head()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,SK_DPD,SK_DPD_DEF
0,100006,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,100011,-38.5,54482.111149,164189.189189,2432.432432,2432.432432,0.0,0.0,3956.221849,4843.064189,4520.067568,52402.088919,54433.179122,54433.179122,0.000000,0.000000
2,100013,-48.5,18159.919219,131718.750000,6350.000000,5953.125000,0.0,0.0,1454.539551,7168.346250,6817.172344,17255.559844,18101.079844,18101.079844,0.010417,0.010417
3,100021,-10.0,0.000000,675000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,100023,-7.5,0.000000,135000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
num_group_df.shape

(103558, 16)

In [17]:
categorical = pd.get_dummies(credit_card_bal[cat_features])
categorical['SK_ID_CURR'] = credit_card_bal['SK_ID_CURR']
categorical.head()

,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed,SK_ID_CURR
0,1,0,0,0,0,0,0,378907
1,1,0,0,0,0,0,0,363914
2,1,0,0,0,0,0,0,371185
3,1,0,0,0,0,0,0,337855
4,1,0,0,0,0,0,0,126868


In [18]:
cat_group_df = categorical.groupby('SK_ID_CURR').agg('sum').reset_index()
cat_group_df.head()

,SK_ID_CURR,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,100006,6,0,0,0,0,0,0
1,100011,74,0,0,0,0,0,0
2,100013,96,0,0,0,0,0,0
3,100021,7,0,10,0,0,0,0
4,100023,8,0,0,0,0,0,0


In [19]:
cat_group_df.shape

(103558, 8)

In [20]:
cnt = credit_card_bal[cnt_col]

In [21]:
cnt['SK_ID_CURR'] = credit_card_bal['SK_ID_CURR']

C:\Users\dbda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
cnt_group_df = cnt.groupby('SK_ID_CURR').agg('sum').reset_index()

In [23]:
cnt_group_df.head()

,SK_ID_CURR,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM
0,100006,0.0,0,0.0,0.0,0.0
1,100011,4.0,4,0.0,0.0,1881.0
2,100013,23.0,23,0.0,0.0,1666.0
3,100021,0.0,0,0.0,0.0,0.0
4,100023,0.0,0,0.0,0.0,0.0


In [24]:
cnt_group_df.shape

(103558, 6)

In [25]:
credit_card_bal = num_group_df.merge(cat_group_df,on='SK_ID_CURR')

In [26]:
credit_card_bal.shape

(103558, 23)

In [27]:
credit_card_bal = credit_card_bal.merge(cnt_group_df,on='SK_ID_CURR')

In [28]:
credit_card_bal.shape

(103558, 28)

In [29]:
credit_card_bal.to_csv('credit_card_bal_aggregate.csv',index=False)